# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [15]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
cf.go_offline()
import xlrd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_excel('../data/Online Retail.xlsx')
df = pd.DataFrame(data)
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [4]:
df1 = df[(df['Country'] != 'United Kingdom') & (df.InvoiceDate > '2011-04-01') & (df.InvoiceDate < '2011-04-30')]
df1.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
724,549667,2011-04-11 12:20:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
3320,550899,2011-04-21 12:07:00,22752,SET 7 BABUSHKA NESTING BOXES,2,8.50,17.00,13505,Switzerland
4880,550527,2011-04-19 10:48:00,84879,ASSORTED COLOUR BIRD ORNAMENT,40,1.69,67.60,12476,Germany
4882,550620,2011-04-19 13:39:00,84879,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,12585,Germany


In [5]:
df2 = df1.groupby(["Country"])[(["Quantity", "Revenue"])].sum()
display(df2)

,Quantity,Revenue
Country,,
Australia,224,421.60
Austria,308,584.78
Belgium,1170,1788.48
Brazil,356,1143.60
Channel Islands,96,243.00
EIRE,4129,7270.50
Finland,810,1368.92
France,2265,3899.31
Germany,5702,10994.79


In [6]:
df2.iplot(kind='bar')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [7]:
df2 = df[(df['Country'] == 'France') & (df.InvoiceDate > '2011-01-01') & (df.InvoiceDate < '2011-05-31')]
df2.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France
3290,545181,2011-02-28 14:45:00,22752,SET 7 BABUSHKA NESTING BOXES,4,8.50,34.00,12509,France
4614,542629,2011-01-31 09:57:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France
4795,548409,2011-03-31 10:27:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France


In [8]:
df3 = df2[(["Quantity", "Revenue"])]
df3.head()

,Quantity,Revenue
807,9,26.55
2645,4,15.00
3290,4,34.00
4614,160,232.00
4795,160,232.00


In [9]:
df3.iplot(kind="lines")

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [38]:
df4 = df[(df['Description'] == 'PARTY BUNTING')]
x = df4['Quantity']
y = df4['UnitPrice']

In [42]:
df4.plot(x, y, kind="scatter")

KeyError: "None of [Int64Index([ 5,  5,  2,  1,  5,  1,  5, 50,  5,  1,\n            ...\n             1, 12,  2,  4,  8,  4,  3,  5,  8,  1],\n           dtype='int64', length=1396)] are in the [columns]"

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.